In [ ]:
# !ffmpeg -i zkmultisensor.mp4 -vn -acodec copy zkmultisensor.mp3

In [ ]:
# playing a voice file
# import IPython
# display(IPython.display.Audio('zkmultisensor/Split_120.mp3'))

In [ ]:
# Manual downloading any video
%pip install yt-dlp
%pip install moviepy
import yt_dlp
from moviepy.video.io.VideoFileClip import VideoFileClip

def download_youtube_video_split(url, output_path):
    ydl_opts = {'outtmpl':output_path,'format_sort': ['res:1080', 'ext:mp4:m4a']}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    
    directoryName = output_path.split(".")[0]
    mp3_file = output_path.split(".")[0] + ".mp3"

    videoFile = VideoFileClip(output_path)
    audio_clip = videoFile.audio
    audio_clip.write_audiofile(mp3_file)

    tlen = videoFile.duration 
    print(tlen)
    
    os.makedirs(directoryName, exist_ok=True)
    for i in range(0, int(tlen), 20):
        str_i = str(f'{(i//100)%1000}{(i//10)%10}{i%10}')

        viideoPart = videoFile[i:min(tlen,i+20)]
        viideoPart.write_videofile("{}/Split_{}.mp4".format(directoryName,str_i))
        audioPart = audio_clip[i:min(tlen,i+20)]
        audioPart.write_audiofile("{}/Split_{}.mp3".format(directoryName,str_i))
        
    audio_clip.close()
    videoFile.close()

In [ ]:
## List all the TouTube files here
url1 = "https://www.youtube.com/watch?v=VZMBE2NLSC4&t=5s"
download_youtube_video_split(url1, "zkmultisensor.mp4")

In [ ]:
audio_directory = "zkmultisensor"
myfilenames = sorted(os.listdir(audio_directory))

for filename in myfilenames:
    if filename.endswith(".mp3"):
        filepath = os.path.join(audio_directory, filename)
        os.system(f"ffmpeg -i {filepath} {filepath[:-4]}.wav") 

In [ ]:
%pip install SpeechRecognition

In [ ]:
import speech_recognition as sr
import os

audio_directory = "zkmultisensor"
myfilenames = sorted(os.listdir(audio_directory))
print(myfilenames)

r = sr.Recognizer()
with open('recognized_text.txt', 'w', encoding='utf-8') as output_file:
  for chunk_file in myfilenames:
    print(chunk_file)
    print(chunk_file[-4:])
    if chunk_file[-4:] == ".wav":
      print(chunk_file)
      filepath = os.path.join(audio_directory, chunk_file)
      print(filepath)
      with sr.AudioFile(filepath) as source:
        audio_data = r.record(source)
        try:
          text = r.recognize_google(audio_data, language='en-US')  # Persian language code
          output_file.write(text + '\n')
        except :
          pass

In [ ]:
class Mydocument():
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

    def __str__(self):
        return f"Page content: {self.page_content}\nMetadata: {self.metadata}"
    
    def __repr__(self):
        return self.__str__()

mydoc = []
with open('recognized_text.txt', 'r', encoding='utf-8') as input_file:
    textlines = input_file.readlines()
for i, chunk_file in enumerate(filter(lambda x: x.endswith(".mp4"), myfilenames)):                      
    mydoc.append(Mydocument(textlines[i], {"source":chunk_file}))
    print(chunk_file)
    
print(mydoc)

############################# Creating Vector database
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

api_key1 = "aaaaaaa"
embedding1 = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key1)
# mydb1 = Chroma.from_documents(totaldoc2, embedding=embedding1)

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

alldocs = text_splitter.split_documents(mydoc)
print(alldocs)

#mydb1 = Chroma.from_documents(alldocs, embedding=embedding1)
mydb1 = Chroma(
    collection_name="example_collection",
    embedding_function=embedding1,
    persist_directory="chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

mydb1.add_documents(alldocs)

allok = mydb1.similarity_search("What is zkIoT?", k=1)
print(allok)

